# Virtual Drug Trials: In Silico Treatment Optimization

## Pharmacological Testing Without Patients

This tutorial demonstrates:
1. Drug library (7 FDA-approved GI drugs)
2. Pharmacokinetics (PK) and pharmacodynamics (PD) modeling
3. Single-patient virtual trials
4. Cohort-based clinical trials
5. Dose-response curves
6. Treatment recommendations

**Clinical Applications:**
- Test drug efficacy *before* prescription
- Optimize dosing for individual patients
- Screen for non-responders
- Predict side effects (multi-target drugs)

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import sys

sys.path.insert(0, '..')

from ens_gi_core import ENSGIDigitalTwin
from ens_gi_drug_library import (
    DrugLibrary, apply_drug, VirtualDrugTrial,
    compute_plasma_concentration
)

print("✓ Drug library imported successfully")
print(f"  Available drugs: {len(DrugLibrary.get_all_drugs())}")

## Part 1: Explore the Drug Library

In [ ]:
# List all available drugs
drugs = DrugLibrary.get_all_drugs()

print("📚 ENS-GI Drug Library (FDA-Approved):")
print("  " + "="*80)
print(f"  {'Drug':<20} {'Class':<25} {'Indication':<35}")
print("  " + "-"*80)

for drug in drugs:
    print(f"  {drug.name:<20} {drug.drug_class:<25} {drug.indication:<35}")

print("  " + "="*80)

## Part 2: Pharmacokinetics - Drug Absorption and Clearance

In [ ]:
# Example: Mexiletine pharmacokinetics
mexiletine = DrugLibrary.MEXILETINE

print(f"📊 {mexiletine.name} Pharmacokinetics:")
print(f"  Standard dose: {mexiletine.standard_dose_mg} mg")
print(f"  Bioavailability: {mexiletine.bioavailability*100:.0f}%")
print(f"  Half-life: {mexiletine.half_life_hours:.1f} hours")
print(f"  Volume of distribution: {mexiletine.volume_distribution_L:.0f} L")

# Simulate plasma concentration over time
dose_mg = 200  # Standard dose
times = np.linspace(0, 24, 100)  # 24 hours
concentrations = [compute_plasma_concentration(dose_mg, mexiletine, t) for t in times]

# Plot PK curve
plt.figure(figsize=(10, 6))
plt.plot(times, concentrations, linewidth=2, color='blue')
plt.xlabel('Time (hours)', fontsize=12)
plt.ylabel('Plasma Concentration (µM)', fontsize=12)
plt.title(f'{mexiletine.name} Plasma Concentration (200 mg dose)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Mark half-life
C_max = concentrations[0]
C_half = C_max / 2
plt.axhline(y=C_half, color='red', linestyle='--', alpha=0.5, label=f't₁/₂ = {mexiletine.half_life_hours:.1f} h')
plt.axvline(x=mexiletine.half_life_hours, color='red', linestyle='--', alpha=0.5)
plt.legend(fontsize=11)
plt.show()

print(f"\n  C_max (peak): {concentrations[0]:.2f} µM")
print(f"  C at 12h: {compute_plasma_concentration(dose_mg, mexiletine, 12):.2f} µM")
print(f"  C at 24h: {compute_plasma_concentration(dose_mg, mexiletine, 24):.2f} µM")

## Part 3: Single-Patient Virtual Trial

### Case Study: IBS-C Patient Testing Lubiprostone

In [ ]:
print("📋 CASE: 45-year-old female with IBS-C")
print("  Symptoms: Constipation, infrequent bowel movements")
print("  Prior treatments: Fiber supplements (ineffective)")
print("  Considering: Lubiprostone (Amitiza®) 24 mcg BID\n")

# Create IBS-C patient twin
patient = ENSGIDigitalTwin(n_segments=12)
patient.apply_profile('ibs_c')

# Baseline simulation
print("Running baseline simulation...")
patient.run(2000, dt=0.05, I_stim={5: 10.0}, verbose=False)
baseline_bio = patient.extract_biomarkers()

print("\n📊 Baseline Biomarkers:")
print(f"  Motility Index: {baseline_bio['motility_index']:.3f}")
print(f"  ICC Frequency: {baseline_bio['icc_frequency_cpm']:.2f} cpm")
print(f"  Spike Rate: {baseline_bio['spike_rate_per_neuron']:.3f} Hz")
print(f"  → Diagnosis: IBS-C (hypoexcitable phenotype)")

# Apply Lubiprostone
print("\nApplying Lubiprostone 24 mcg...")
patient_drug = ENSGIDigitalTwin(n_segments=12)
patient_drug.apply_profile('ibs_c')

drug_info = apply_drug(
    patient_drug,
    DrugLibrary.LUBIPROSTONE,
    dose_mg=0.024,  # 24 mcg
    time_hours=2.0
)

print(f"  Plasma concentration: {drug_info['concentration_uM']:.4f} µM")
print(f"  Targets: {drug_info['modified_parameters']}")

# Post-drug simulation
patient_drug.run(2000, dt=0.05, I_stim={5: 10.0}, verbose=False)
drug_bio = patient_drug.extract_biomarkers()

# Compare results
print("\n📊 Post-Treatment Biomarkers:")
print("  Biomarker            Baseline    Post-Drug   Change")
print("  " + "-"*58)

metrics = ['motility_index', 'icc_frequency_cpm', 'spike_rate_per_neuron']
for metric in metrics:
    baseline = baseline_bio[metric]
    post = drug_bio[metric]
    change = (post - baseline) / baseline * 100 if baseline != 0 else 0
    print(f"  {metric:<20} {baseline:>10.3f}  {post:>10.3f}   {change:>+6.1f}%")

# Clinical decision
improvement = (drug_bio['motility_index'] - baseline_bio['motility_index']) / baseline_bio['motility_index']

print("\n💊 Clinical Recommendation:")
if improvement > 0.20:
    print("  ✓ STRONG RESPONDER: Prescribe Lubiprostone 24 mcg BID")
    print("    Expected benefit: Significant improvement in bowel transit")
elif improvement > 0.10:
    print("  ~ MODERATE RESPONDER: Trial Lubiprostone, monitor response")
    print("    Consider dose adjustment or combination therapy")
else:
    print("  ✗ POOR RESPONDER: Consider alternative therapy")
    print("    Alternatives: Linaclotide, Prucalopride")

## Part 4: Compare Multiple Drugs for IBS-D

In [ ]:
print("📋 CASE: IBS-D Patient - Which drug is best?\n")

# Create IBS-D baseline
ibs_d_patient = ENSGIDigitalTwin(n_segments=10)
ibs_d_patient.apply_profile('ibs_d')
ibs_d_patient.run(1500, dt=0.05, verbose=False)
baseline = ibs_d_patient.extract_biomarkers()

# Test multiple drugs
drugs_to_test = [
    ('Ondansetron', DrugLibrary.ONDANSETRON, 8),  # 8 mg
    ('Alosetron', DrugLibrary.ALOSETRON, 1),      # 1 mg
    ('Rifaximin', DrugLibrary.RIFAXIMIN, 550),    # 550 mg
]

results = {}

for drug_name, drug, dose in drugs_to_test:
    twin_drug = ENSGIDigitalTwin(n_segments=10)
    twin_drug.apply_profile('ibs_d')
    apply_drug(twin_drug, drug, dose_mg=dose)
    twin_drug.run(1500, dt=0.05, verbose=False)
    results[drug_name] = twin_drug.extract_biomarkers()

# Compare motility reduction
print("📊 Drug Comparison (IBS-D Treatment):")
print("  Drug              Dose      Motility Reduction   Efficacy")
print("  " + "-"*65)

for drug_name, drug, dose in drugs_to_test:
    reduction = (baseline['motility_index'] - results[drug_name]['motility_index']) / baseline['motility_index'] * 100
    efficacy = "High" if reduction > 20 else "Moderate" if reduction > 10 else "Low"
    print(f"  {drug_name:<16}  {dose:>4} mg   {reduction:>6.1f}%               {efficacy}")

# Visualize
drug_names = [name for name, _, _ in drugs_to_test]
motility_values = [results[name]['motility_index'] for name in drug_names]
motility_values.insert(0, baseline['motility_index'])
labels = ['Baseline'] + drug_names

plt.figure(figsize=(10, 6))
colors = ['red'] + ['green', 'blue', 'orange']
bars = plt.bar(labels, motility_values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
plt.ylabel('Motility Index', fontsize=12)
plt.title('IBS-D Drug Efficacy Comparison', fontsize=14, fontweight='bold')
plt.grid(True, axis='y', alpha=0.3)

# Add value labels
for bar, val in zip(bars, motility_values):
    plt.text(bar.get_x() + bar.get_width()/2, val, f'{val:.3f}',
             ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

print("\n💊 Recommendation: Alosetron shows strongest effect (5-HT3 antagonist)")

## Part 5: Dose-Response Curve

In [ ]:
# Generate dose-response curve for Mexiletine in IBS-C
print("Generating dose-response curve for Mexiletine...\n")

doses = [0, 50, 100, 150, 200, 250, 300]  # mg
responses = []

for dose in doses:
    twin = ENSGIDigitalTwin(n_segments=8)
    twin.apply_profile('ibs_c')
    
    if dose > 0:
        apply_drug(twin, DrugLibrary.MEXILETINE, dose_mg=dose)
    
    twin.run(1000, dt=0.1, verbose=False)
    bio = twin.extract_biomarkers()
    responses.append(bio['motility_index'])
    print(f"  Dose {dose:>3} mg → Motility = {bio['motility_index']:.3f}")

# Plot dose-response
plt.figure(figsize=(10, 6))
plt.plot(doses, responses, 'o-', linewidth=2, markersize=10, color='purple')
plt.xlabel('Mexiletine Dose (mg)', fontsize=12)
plt.ylabel('Motility Index', fontsize=12)
plt.title('Dose-Response Curve: Mexiletine in IBS-C', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

# Find optimal dose (maximum response)
optimal_idx = np.argmax(responses)
optimal_dose = doses[optimal_idx]
optimal_response = responses[optimal_idx]

plt.axvline(x=optimal_dose, color='red', linestyle='--', alpha=0.5,
            label=f'Optimal dose = {optimal_dose} mg')
plt.axhline(y=optimal_response, color='red', linestyle='--', alpha=0.5)
plt.legend(fontsize=11)
plt.show()

print(f"\n📊 Optimal Dose: {optimal_dose} mg")
print(f"   Maximum Response: {optimal_response:.3f}")
print(f"   Improvement over baseline: {(optimal_response - responses[0])/responses[0]*100:.1f}%")

## Part 6: Full Virtual Clinical Trial (Cohort-Based)

In [ ]:
print("\n" + "="*70)
print("VIRTUAL CLINICAL TRIAL: PRUCALOPRIDE FOR IBS-C")
print("="*70)
print("\nTrial Design:")
print("  • Drug: Prucalopride (5-HT4 agonist)")
print("  • Indication: IBS-C")
print("  • Cohort size: 30 patients")
print("  • Doses: 1 mg, 2 mg, 4 mg")
print("  • Duration: 1000 ms simulation\n")

# Create virtual trial
trial = VirtualDrugTrial(
    drug=DrugLibrary.PRUCALOPRIDE,
    cohort_size=30,
    patient_profile='ibs_c',
    parameter_variation=0.15
)

# Run trial (this may take a few minutes)
print("Running virtual trial (this will take 2-3 minutes)...\n")
trial_results = trial.run_trial(
    doses_mg=[1, 2, 4],
    simulation_duration=1000
)

# Print results
VirtualDrugTrial.print_results(trial_results)

# Visualize dose-response
doses_tested = list(trial_results.dose_response_curve.keys())
responses = list(trial_results.dose_response_curve.values())

plt.figure(figsize=(10, 6))
plt.plot(doses_tested, responses, 'o-', linewidth=2, markersize=12, color='green')
plt.xlabel('Prucalopride Dose (mg)', fontsize=12)
plt.ylabel('Mean Response (Δ Motility)', fontsize=12)
plt.title('Virtual Trial: Prucalopride Dose-Response in IBS-C Cohort (n=30)',
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

print("\n💊 Trial Conclusion:")
if trial_results.responder_rate > 0.5:
    print(f"  ✓ POSITIVE TRIAL: {trial_results.responder_rate*100:.0f}% responder rate")
    print(f"  ✓ Recommend: Prucalopride for IBS-C treatment")
else:
    print(f"  ✗ NEGATIVE TRIAL: {trial_results.responder_rate*100:.0f}% responder rate")
    print(f"  ✗ Consider alternative therapies")

## Part 7: Multi-Target Drug Effects

In [ ]:
# Alosetron has multiple targets (5-HT3 receptor + excitatory synapses)
alosetron = DrugLibrary.ALOSETRON

print(f"📊 {alosetron.name} - Multi-Target Analysis")
print(f"  Number of targets: {len(alosetron.targets)}\n")

for idx, target in enumerate(alosetron.targets, 1):
    print(f"  Target {idx}: {target.parameter}")
    print(f"    Type: {target.target_type.value}")
    print(f"    IC50/EC50: {target.ic50_or_ec50:.2f} µM")
    print(f"    Efficacy: {target.efficacy*100:.0f}%")
    print()

print("  Clinical Implication:")
print("  → Dual mechanism: Blocks serotonin AND reduces excitatory transmission")
print("  → Stronger effect than single-target drugs")
print("  → Higher risk of side effects (constipation, ischemic colitis)")

## Summary

This tutorial demonstrated:

1. ✅ **Drug Library** - 7 FDA-approved GI drugs with PK/PD models
2. ✅ **Pharmacokinetics** - Plasma concentration curves
3. ✅ **Single-Patient Trials** - Personalized drug response
4. ✅ **Drug Comparison** - Selecting optimal therapy
5. ✅ **Dose-Response Curves** - Finding optimal dosing
6. ✅ **Cohort Trials** - Statistical analysis (n=30 patients)
7. ✅ **Multi-Target Effects** - Complex drug mechanisms

### Clinical Advantages:

- **Pre-screening**: Identify non-responders before prescription
- **Dose Optimization**: Find patient-specific optimal dose
- **Drug Selection**: Compare efficacy across multiple options
- **Safety**: Predict side effects from multi-target drugs
- **Cost Savings**: Avoid ineffective treatments

### Available Drugs:

1. **Mexiletine** - Na+ blocker (off-label for IBS)
2. **Ondansetron** - 5-HT3 antagonist (IBS-D)
3. **Alosetron** - 5-HT3 antagonist (severe IBS-D)
4. **Lubiprostone** - ClC-2 activator (IBS-C)
5. **Linaclotide** - Guanylate cyclase agonist (IBS-C)
6. **Prucalopride** - 5-HT4 agonist (IBS-C)
7. **Rifaximin** - Antibiotic (IBS with bloating)

### Next Steps:

- Add more drugs to library (custom drug profiles)
- Model drug-drug interactions
- Integrate with real patient data
- Prospective clinical validation